In [29]:
import nltk
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
from scipy.special import softmax
from collections import Counter




In [7]:
DATA_PATH = '../../data/wiki'
k=0.5
col='label'
txt='text'
df = pd.read_csv(f'{DATA_PATH}/train_original_split.csv')


In [11]:
list_of_classes = list(set(df[col]))
NUM_OF_CLASSES = len(list_of_classes)
print('---Number of Classes---')
print(NUM_OF_CLASSES)
all_sents = list(df['text'])
all_tokenized_sents = [word_tokenize(s) for s in all_sents]
stop_words = set(stopwords.words('english'))
sentences_stopped = [[x for x in y if not x.lower() in stop_words]for y in all_tokenized_sents]

vocab_pre_stem = list(set([word for s in sentences_stopped for word in s]))
print('---Vocab Size Pre Stemming---')
print(len(vocab_pre_stem))

ps = PorterStemmer()

print('---Building Inverse Stemming Lookup---')
inverse_stem_lookup = defaultdict(set)
for v in vocab_pre_stem:
    inverse_stem_lookup[ps.stem(v)].add(v)

all_stemmed_sents = [[ps.stem(w) for w in s] for s in all_tokenized_sents]
vocab_post_stem = list(set([word for s in all_stemmed_sents for word in s]))

print('---Vocab Size Post Stemming---')
VOCAB_SIZE = len(vocab_post_stem)
print(VOCAB_SIZE)

---Number of Classes---
2
---Vocab Size Pre Stemming---
83022
---Building Inverse Stemming Lookup---
---Vocab Size Post Stemming---
55349


In [15]:
vocab_post_stem[:10]

['neurographi',
 '1942',
 'id=ncu8snx2cgec',
 'outweight',
 'coddl',
 '4/22',
 'x',
 'dvdm',
 'lphant',
 'artistri']

In [23]:
df[df[txt].str.contains('neurographi')]



,text,label


In [28]:
df['tokens_full'] = df.apply(lambda x: word_tokenize(x[txt]), axis=1)
df['tokens_clean'] = df.apply(lambda x: [w.lower() for w in x['tokens_full'] if w not in stop_words and w.isalnum()], axis=1)
df['tokens_stem'] = df.apply(lambda x: [ps.stem(w) for w in x['tokens_clean']], axis=1)
df

,text,label,tokens_full,tokens_clean,tokens_stem
0,:If I may butt in I've spent the last 1/4 hour...,0,"[:, If, I, may, butt, in, I, 've, spent, the, ...","[if, i, may, butt, i, spent, last, hour, follo...","[if, i, may, butt, i, spent, last, hour, follo..."
1,On my you will find the apology that I owe you...,0,"[On, my, you, will, find, the, apology, that, ...","[on, find, apology, i, owe, shuffles, feet, lo...","[on, find, apolog, i, owe, shuffl, feet, look,..."
2,"Yep, that's Twin cities from which this articl...",0,"[Yep, ,, that, 's, Twin, cities, from, which, ...","[yep, twin, cities, article, originated]","[yep, twin, citi, articl, origin]"
3,See? I was right! ;-),0,"[See, ?, I, was, right, !, ;, -, )]","[see, i, right]","[see, i, right]"
4,` Thanks for fixing the spelling error in ``pr...,0,"[`, Thanks, for, fixing, the, spelling, error,...","[thanks, fixing, spelling, error, propaganda, ...","[thank, fix, spell, error, propaganda, i, ca, ..."
...,...,...,...,...,...
23134,` kys {| style=``background-color: #fdffe7; bo...,0,"[`, kys, {, |, style=, ``, background-color, :...","[kys, fdffe7, border, 1px, solid, fceb92, 2, m...","[ky, fdffe7, border, 1px, solid, fceb92, 2, mi..."
23135,"Yeah, I realized I created a duplicate ID. Sor...",0,"[Yeah, ,, I, realized, I, created, a, duplicat...","[yeah, i, realized, i, created, duplicate, id,...","[yeah, i, realiz, i, creat, duplic, id, sorri,..."
23136,` :::Yeah and in the earlier sentence I'd rewo...,0,"[`, :, :, :, Yeah, and, in, the, earlier, sent...","[yeah, earlier, sentence, i, reword, fever, pi...","[yeah, earlier, sentenc, i, reword, fever, pitch]"
23137,Why oh why... You removed the trolls ANI secti...,0,"[Why, oh, why, ..., You, removed, the, trolls,...","[why, oh, you, removed, trolls, ani, section, ...","[whi, oh, you, remov, troll, ani, section, drm..."


In [52]:
tokens = [x for y in df['tokens_stem'] for x in y]
count = Counter(tokens)
count_class = Counter(df['label'])

VOCAB_SIZE = len(count.keys())
NUM_OF_TOKENS = sum(count.values())
NUM_OF_CLASSES = len(count_class.keys())
NUM_OF_RECORDS = sum(count_class.values())

In [36]:
c = 0
for k, v in count.items():
    if v == 1:
        c +=1
        
len(count)

40553

In [47]:
sum(count_class.values())
len(df)

23139

In [55]:
likelihood = np.zeros((VOCAB_SIZE, NUM_OF_CLASSES))
prior = np.zeros((NUM_OF_CLASSES,1))
norm = np.zeros((VOCAB_SIZE, 1))

In [62]:
vocab = list(count.keys())
classes = list(count_class.keys())
for k, v in count_class.items():
    sub_df = df[df['label']==k]
    sub_tokens = [x for y in sub_df['tokens_stem'] for x in y]
    prior[classes.index(k), 0] = v / len(df)
    sub_token_count = Counter(sub_tokens)
    for s_token, s_count in sub_token_count.items():
        likelihood[vocab.index(s_token), classes.index(k)] = s_count / sum(sub_token_count.values())
        
for k, v in count.items():
    norm[vocab.index(k), 0] = v / NUM_OF_TOKENS


In [63]:
likelihood * np.repeat(prior,VOCAB_SIZE,axis=1).T / np.repeat(norm, NUM_OF_CLASSES, axis=1)

array([[0.90466626, 0.09664234],
       [0.8807418 , 0.11923531],
       [0.98000006, 0.02550114],
       ...,
       [1.00700407, 0.        ],
       [0.        , 0.95096063],
       [1.00700407, 0.        ]])

In [59]:
NUM_OF_TOKENS

915554